# Segmenting and Clustering Neighborhoods in Toronto - Explore and Cluster

In [1]:
import pandas as pd

In [2]:
latlong = pd.read_csv("http://cocl.us/Geospatial_data/") 
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
#Rename "Postal Code" with "PostalCode"
latlong.columns = ['PostalCode','Latitude','Longitude']
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Make sure there are 103 postal codes as we determined from previous assignment

In [4]:
latlong.shape[0]

103

### We use the code from the Notebook SCN-Toronto-Scrape-Wiki.ipynb to get the postcode, borough and neigborhood (previous assignment)

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False) 
tdf = table[0]
tdf.columns = ['PostalCode','Borough','Neighborhood']
tdf = tdf.query('Borough != "Not assigned"').reset_index(drop=True)
tdf1=tdf.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
tdf1.loc[tdf1['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = tdf1['Borough']
tdf1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### We merge the two tables with latlong on the right

In [7]:
tdf2 = pd.merge(left=tdf1, right=latlong, on="PostalCode", how="right")
tdf2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Let us now explore Toronto

### Map of Toronto with neighborhoods superimposed

In [12]:
!conda install -c conda-forge folium=0.5.0 --yes

import folium 

# create map of York using latitude and longitude values

map_Toronto = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# add markers to map
for lat, lng, bor, nei in zip(tdf2['Latitude'], tdf2['Longitude'], tdf2['Borough'], tdf2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

### Define Foursquare Credentials and Version

In [13]:
###Removing CLIENT_ID and CLIENT_SECRET after run

CLIENT_ID = '1XJRYLTACKK43ZQRBM2BI41HXPBWLCPJSEDM0YWT41ROYKTI' # Foursquare ID
CLIENT_SECRET = '2XDNXZSA2QCSQJYHXG02X4EKF3KS4CAD1TZD5KRDHD5GGFSI' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Examine Neighborhoods in Scarborough, lat-long 43.773077, -79.257774

In [14]:
Scarborough_data = tdf2[tdf2['Borough'] == 'Scarborough'].reset_index(drop=True)
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

### We find nearby venues in Scarborough

In [15]:
import requests
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Malvern, Rouge
Port Union, Rouge Hill, Highland Creek
Guildwood, West Hill, Morningside
Woburn
Cedarbrae
Scarborough Village
Ionview, Kennedy Park, East Birchmount Park
Clairlea, Golden Mile, Oakridge
Cliffside, Scarborough Village West, Cliffcrest
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Sullivan, Clarks Corners, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


## Now we cluster and visulaize neighborhoods

In [16]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffside, Scarborough Village West, Cliffcrest",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Guildwood, West Hill, Morningside",9,9,9,9,9,9
"Ionview, Kennedy Park, East Birchmount Park",4,4,4,4,4,4


In [17]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.250000,0.0,...,0.000000,0.250000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.333333,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000,0.000000,0.000000
3,Cedarbrae,0.000000,0.000000,0.125,0.000000,0.125000,0.125,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.000,0.000000,0.200000,0.000,0.0,0.000000,0.2,...,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000,0.000,0.000000,0.000000
5,"Cliffside, Scarborough Village West, Cliffcrest",0.333333,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
6,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.166667
7,"Guildwood, West Hill, Morningside",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.111111,0.0,...,0.111111,0.000000,0.000000,0.00,0.0,0.111111,0.111111,0.000,0.000000,0.000000
8,"Ionview, Kennedy Park, East Birchmount Park",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
9,L'Amoreaux West,0.083333,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.083333,0.0,...,0.000000,0.083333,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.083333,0.000000


In [18]:
import numpy as np
num_top_venues = 17

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Caribbean Restaurant,Hakka Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Coffee Shop,Caribbean Restaurant,Chinese Restaurant,Hakka Restaurant
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Caribbean Restaurant,Chinese Restaurant,Indian Restaurant
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Vietnamese Restaurant
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Chinese Restaurant
5,"Cliffside, Scarborough Village West, Cliffcrest",Motel,American Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
6,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Bakery,Grocery Store,General Entertainment,Fried Chicken Joint,Asian Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Auto Garage
7,"Guildwood, West Hill, Morningside",Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium,Convenience Store,Construction & Landscaping,Chinese Restaurant,Department Store,Discount Store,Fast Food Restaurant,Coffee Shop
8,"Ionview, Kennedy Park, East Birchmount Park",Discount Store,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,Construction & Landscaping,College Stadium,Caribbean Restaurant,Indian Restaurant,Café
9,L'Amoreaux West,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Breakfast Spot,American Restaurant,Thrift / Vintage Store,Sandwich Place,Department Store,Discount Store,Convenience Store,Construction & Landscaping,Electronics Store,Fried Chicken Joint,College Stadium


### Cluster the neighborhood Scarborough

In [19]:
from sklearn.cluster import KMeans


# Number of clusters 4
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 3, 1, 1, 0], dtype=int32)

In [20]:
scarborough_merged = Scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


scarborough_merged

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,...,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Café
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,...,Pizza Place,Rental Car Location,College Stadium,Convenience Store,Construction & Landscaping,Chinese Restaurant,Department Store,Discount Store,Fast Food Restaurant,Coffee Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Construction & Landscaping,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,...,Hakka Restaurant,College Stadium,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Vietnamese Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,...,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Café
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029,1,Discount Store,Department Store,Playground,Coffee Shop,...,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,Construction & Landscaping,College Stadium,Caribbean Restaurant,Indian Restaurant,Café
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Intersection,Fast Food Restaurant,...,Park,College Stadium,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Chinese Restaurant
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476,0,Motel,American Restaurant,Coffee Shop,Grocery Store,...,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,...,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Caribbean Restaurant,Chinese Restaurant,Indian Restaurant


In [21]:

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.773077, -79.257774], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


In [22]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
1,"Port Union, Rouge Hill, Highland Creek",0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
8,"Cliffside, Scarborough Village West, Cliffcrest",0,Motel,American Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
15,L'Amoreaux West,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Breakfast Spot,American Restaurant,Thrift / Vintage Store,Sandwich Place,Department Store,Discount Store,Convenience Store,Construction & Landscaping,Electronics Store,Fried Chicken Joint,College Stadium


### cluster1

In [23]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,"Malvern, Rouge",1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Café
2,"Guildwood, West Hill, Morningside",1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium,Convenience Store,Construction & Landscaping,Chinese Restaurant,Department Store,Discount Store,Fast Food Restaurant,Coffee Shop
3,Woburn,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Construction & Landscaping,College Stadium,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Café
4,Cedarbrae,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Vietnamese Restaurant
6,"Ionview, Kennedy Park, East Birchmount Park",1,Discount Store,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,Construction & Landscaping,College Stadium,Caribbean Restaurant,Indian Restaurant,Café
7,"Clairlea, Golden Mile, Oakridge",1,Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Chinese Restaurant
9,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Caribbean Restaurant,Chinese Restaurant,Indian Restaurant
10,"Dorset Park, Wexford Heights, Scarborough Town...",1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Bakery,Grocery Store,General Entertainment,Fried Chicken Joint,Asian Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Auto Garage
11,"Wexford, Maryvale",1,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,Caribbean Restaurant
13,"Sullivan, Clarks Corners, Tam O'Shanter",1,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank,Chinese Restaurant,Pharmacy,Coffee Shop,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Convenience Store,Department Store,Electronics Store,Discount Store


### cluster2

In [24]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
5,Scarborough Village,2,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Café


### cluster3

In [25]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
12,Agincourt,3,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium,Caribbean Restaurant,Hakka Restaurant
